In [ ]:
import cv2
import numpy as np
import csv


bg_subtractor = cv2.createBackgroundSubtractorMOG2()

CSV_FILENAME = "fall_detection_features.csv"

with open(CSV_FILENAME, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Video", "Frame", "Bh", "Bw", "Br", "Cx", "Cy", "Ex", "Ey", "Hph", "Vph", "m00", "m11", "m02", "m20", "Eo"])

def extract_features(frame, bounding_box):
    x, y, w, h = bounding_box
    aspect_ratio = h / (w + 1e-5)  # Avoid division by zero
    cx, cy = x + w // 2, y + h // 2  # Center of bounding box
    
    # Fit ellipse if possible
    mask = np.zeros_like(frame, dtype=np.uint8)
    mask[y:y+h, x:x+w] = 255
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours and len(contours[0]) > 4:
        ellipse = cv2.fitEllipse(contours[0])
        ex, ey = ellipse[0]
        orientation = ellipse[2]
    else:
        ex, ey, orientation = cx, cy, 0
    
    # Histogram projections
    h_proj = np.sum(mask, axis=1)
    v_proj = np.sum(mask, axis=0)
    
    # Compute moments
    M = cv2.moments(mask, binaryImage=True)
    m00 = M['m00']
    m11 = M['m11'] / (m00 + 1e-5)
    m02 = M['m02'] / (m00 + 1e-5)
    m20 = M['m20'] / (m00 + 1e-5)
    
    return [h, w, aspect_ratio, cx, cy, ex, ey, np.sum(h_proj), np.sum(v_proj), m00, m11, m02, m20, orientation]


In [ ]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    video_name = video_path.split('/')[-1]  # Extract video filename
    frame_count = 0
    
    with open(CSV_FILENAME, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            fg_mask = bg_subtractor.apply(gray)
            
            contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                max_contour = max(contours, key=cv2.contourArea)
                bounding_box = cv2.boundingRect(max_contour)
                features = extract_features(gray, bounding_box)
                writer.writerow([video_name, frame_count] + features)
            
            frame_count += 1
    
    cap.release()
    print(f"Processed {video_name} and saved results to {CSV_FILENAME}")


In [ ]:
video_files = ["fall_video_1.mp4", "fall_video_2.mp4"] 
for video in video_files:
    process_video(video)